<a href="https://colab.research.google.com/github/bksat90/FinalProject/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installation of KerasCV.
!pip install -q --upgrade keras-cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.9 MB/s eta 0:00:00


In [5]:
# Importing relevant modules.
import os
import tqdm
import numpy as np

os.environ["KERAS_BACKEND"] = "tensorflow"
from tensorflow import data as tf_data
import tensorflow_datasets as tfds
import keras
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
import xml.etree.ElementTree as ET
import tensorflow as tf

In [3]:
# Import and mount Google Drive.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# # Unzip the dataset file and store it in a folder called images.
# !unzip "/content/drive/MyDrive/Project_BC/data_voc.zip" -d "/content/drive/MyDrive/Project_BC/data/"

Archive:  /content/drive/MyDrive/Project_BC/data_voc.zip
 extracting: /content/drive/MyDrive/Project_BC/data/README.dataset.txt  
 extracting: /content/drive/MyDrive/Project_BC/data/README.roboflow.txt  
   creating: /content/drive/MyDrive/Project_BC/data/test/
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00037_RIGHT_CC_png.rf.3ae0fe363a68b249aa3d0051ed5cf4f8.jpg  
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00037_RIGHT_CC_png.rf.3ae0fe363a68b249aa3d0051ed5cf4f8.xml  
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00099_LEFT_MLO_png.rf.7434a1e263cb91a816e32085a7dbfe5b.jpg  
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00099_LEFT_MLO_png.rf.7434a1e263cb91a816e32085a7dbfe5b.xml  
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00114_LEFT_MLO_png.rf.d468144d3e75f286c0a617620e573916.jpg  
 extracting: /content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00114_LEFT_MLO_png.

In [8]:
class_ids = [
    "benign",
    "malignant",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
train_path = "/content/drive/MyDrive/Project_BC/data/train/"
test_path = "/content/drive/MyDrive/Project_BC/data/test/"

# Get all XML file paths in path_annot and sort them
train_xml_files = sorted(
    [
        os.path.join(train_path, file_name)
        for file_name in os.listdir(train_path)
        if file_name.endswith(".xml")
    ]
)

# Get all train JPEG image file paths in path_images and sort them
train_jpg_files = sorted(
    [
        os.path.join(train_path, file_name)
        for file_name in os.listdir(train_path)
        if file_name.endswith(".jpg")
    ]
)

test_xml_files = sorted(
    [
        os.path.join(test_path, file_name)
        for file_name in os.listdir(train_path)
        if file_name.endswith(".xml")
    ]
)

# Get all test JPEG image file paths in path_images and sort them
test_jpg_files = sorted(
    [
        os.path.join(test_path, file_name)
        for file_name in os.listdir(train_path)
        if file_name.endswith(".jpg")
    ]
)

In [9]:
test_xml_files

['/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00016_LEFT_CC_png.rf.e457e087985346df43e82f4a66f18a99.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00016_LEFT_MLO_png.rf.b27cd999a4b4fcc2f11ec51274da67b5.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00017_LEFT_CC_png.rf.b175e7dfba2a66e020484f4001280c66.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00017_LEFT_MLO_png.rf.dfab9e0967d3bc547930fd2bb4b8a335.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00032_RIGHT_CC_png.rf.1e80492e83d7a81b8594616ca69cbc4e.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00032_RIGHT_MLO_png.rf.274f64ab2e7bb5742d223236dca388df.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00037_RIGHT_MLO_png.rf.857b6096e2afd49a7d398b0a46745330.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Mass-Test_P_00056_LEFT_MLO_png.rf.4544d7f3e64955d174709d1d48844f05.xml',
 '/content/drive/MyDrive/Project_BC/data/test/Ma